<a href="https://colab.research.google.com/github/bogdanbabych/experiments_NLTK/blob/main/Intelligence_Task_Ontology_(ITO)_exploration_and_visualization_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install Owlready2
from owlready2 import *

     |████████████████████████████████| 20.8MB 1.4MB/s 
  Created wheel for Owlready2: filename=Owlready2-0.24-cp36-cp36m-linux_x86_64.whl size=20262991 sha256=eac0bba5e80a337aaa2a062d17f82222407226d72257b324311153f254c38908
  Stored in directory: /root/.cache/pip/wheels/66/87/0b/7a97bd19c9dc817b279aadb5dd920ce3a77050e2ab5a413f43
Successfully built Owlready2


In [ ]:
#onto = get_ontology("file:///content/ITO.owl")
onto = get_ontology("https://github.com/OpenBioLink/ITO/raw/master/ITO.owl")
onto.load()
ITO = onto.get_namespace("https://ai-strategies.org/ontology/")
EDAM = onto.get_namespace("http://edamontology.org/")

# Creating ontology statistics

Please note that the queries in this example Colab notebook are run in the notebook with a slow Python RDF store, and more complex queries can have a long runtime. Use a performant SPARQL store such as https://blazegraph.com/ for fast queries (which can be queried with Python through the https://github.com/RDFLib/sparqlwrapper library).

In [ ]:
! pip install rdflib
import rdflib
graph = default_world.as_rdflib_graph()

In [ ]:
import pandas as pd
import json
def sparql_result_to_df(sparql_result):
    # converts SPARQL SELECT results to Pandas dataframe
    
    cols = sparql_result.bindings[0].keys()

    out = []
    for row in sparql_result.bindings:
        item = []
        for c in cols:
            item.append(row.get(c, {}))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

In [ ]:
namespaces ={ 'owl': rdflib.OWL,
              'rdfs': rdflib.RDFS,
              'edam': 'http://edamontology.org/',
              'ito': 'https://ai-strategies.org/ontology/'}

### Are there entities/properties with more than one label? (data quality check)

In [ ]:
result = graph.query("""
SELECT ?entity 
       (COUNT(?entity_label) AS ?entity_label_count)
WHERE {
    ?entity rdfs:label ?entity_label .
}
GROUP BY ?entity
ORDER BY DESC(?entity_label_count)

""", initNs = namespaces)
df = sparql_result_to_df(result)
df

,entity,entity_label_count
0,https://ai-strategies.org/ontology,2
1,http://edamontology.org/citation,1
2,http://edamontology.org/documentation,1
3,http://edamontology.org/example,1
4,http://edamontology.org/file_extension,1
...,...,...
41053,https://ai-strategies.org/ontology/ito_172696,1
41054,https://ai-strategies.org/ontology/ito_172697,1
41055,https://ai-strategies.org/ontology/ito_172698,1
41056,https://ai-strategies.org/ontology/ito_172881,1


### Property stats on individual level (how many measurements with each property?)

In [ ]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark_process_indivdual) AS ?benchmark_process_indivdual_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?benchmark_process_indivdual ?performance_measure ?result .
}
GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_process_indivdual_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('direct_property_statistics.csv', index=False)
df

,performance_measure_label,performance_measure,benchmark_process_indivdual_count
0,Accuracy,https://ai-strategies.org/ontology/ITO_02411,5284
1,Score,https://ai-strategies.org/ontology/ITO_08816,3742
2,Accuracy (CS),https://ai-strategies.org/ontology/ITO_06723,1230
3,Accuracy (CV),https://ai-strategies.org/ontology/ITO_06722,1107
4,F1,https://ai-strategies.org/ontology/ITO_03697,985
...,...,...,...
793,Test error,https://ai-strategies.org/ontology/ITO_17265,1
794,BPE Perplexity,https://ai-strategies.org/ontology/ITO_10156,1
795,Consistency,https://ai-strategies.org/ontology/ITO_15741,1
796,Chamfer Distance,https://ai-strategies.org/ontology/ITO_15903,1


In [ ]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark_process_indivdual_in_subproperty) AS ?benchmark_process_individual_in_subproperty_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?performance_measure_subproperty rdfs:subPropertyOf* ?performance_measure .
    ?benchmark_process_indivdual_in_subproperty ?performance_measure_subproperty ?result_s .
}

GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_process_individual_in_subproperty_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('subproperty_statistics.csv', index=False)
df

,performance_measure_label,performance_measure,benchmark_process_individual_in_subproperty_count
0,Accuracy,https://ai-strategies.org/ontology/ITO_01157,11790
1,Score,https://ai-strategies.org/ontology/ITO_08816,3752
2,Precision,https://ai-strategies.org/ontology/ITO_04547,3336
3,F1 score NEW,https://ai-strategies.org/ontology/ITO_00001,1887
4,Peak signal-to-noise ratio,https://ai-strategies.org/ontology/ITO_00016,944
...,...,...,...
182,CREMI Score,https://ai-strategies.org/ontology/ITO_05631,1
183,BPE Perplexity,https://ai-strategies.org/ontology/ITO_10156,1
184,Consistency,https://ai-strategies.org/ontology/ITO_15741,1
185,Chamfer Distance,https://ai-strategies.org/ontology/ITO_15903,1


### Property stats on benchmark level (how many benchmarks with each property?)

In [ ]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark) AS ?benchmark_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?benchmark rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure
}
GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('direct_property_statistics_per_benchmark.csv', index=False)
df

,performance_measure_label,performance_measure,benchmark_count
0,Accuracy,https://ai-strategies.org/ontology/ITO_02411,583
1,F1,https://ai-strategies.org/ontology/ITO_03697,137
2,Score,https://ai-strategies.org/ontology/ITO_08816,98
3,AP,https://ai-strategies.org/ontology/ITO_02616,78
4,AUC,https://ai-strategies.org/ontology/ITO_02593,75
...,...,...,...
807,Average Treatment Effect Error,https://ai-strategies.org/ontology/ITO_15752,1
808,Mean target overlap ratio,https://ai-strategies.org/ontology/ITO_15853,1
809,Chamfer Distance,https://ai-strategies.org/ontology/ITO_15903,1
810,MACE,https://ai-strategies.org/ontology/ITO_15994,1


## Total number of benchmarks

In [ ]:
result = graph.query("""
SELECT (COUNT(?benchmark) as ?benchmark_count) WHERE {
    ?benchmark rdfs:subClassOf ito:Benchmarking
}
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('total_number_of_benchmarks.csv', index=False)
df

,benchmark_count
0,2298


## Number of different metrics per benchmark (for those where any metrics are listed) 



In [ ]:
result = graph.query("""
SELECT ?benchmark_label ?benchmark
       (COUNT(DISTINCT?performance_measure) AS ?performance_measure_count)
       (GROUP_CONCAT(DISTINCT ?performance_measure_label;SEPARATOR=";") AS ?performance_measure_labels)
WHERE {
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?benchmark rdfs:subClassOf ito:Benchmarking ;
              rdfs:label ?benchmark_label ;
              rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure .
}
GROUP BY ?benchmark_label ?benchmark
ORDER BY DESC(?performance_measure_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number-of-different-metrics-per-benchmark.csv', index=False)
df

,benchmark_label,benchmark,performance_measure_count,performance_measure_labels
0,MSR-VTT - Video Retrieval benchmarking,https://ai-strategies.org/ontology/ITO_07273,10,text-to-video Median Rank;text-to-video R-at-1...
1,COCO test-dev - Keypoint Detection benchmarking,https://ai-strategies.org/ontology/ITO_11334,10,AR;AR50;AR75;ARM;AP;AP50;AP75;APM;APL;ARL
2,"MIT-States, generalized split - Compositional ...",https://ai-strategies.org/ontology/ITO_07763,9,H-Mean;Test AUC top 1;Test AUC top 2;Test AUC ...
3,COCO test-challenge - Keypoint Detection bench...,https://ai-strategies.org/ontology/ITO_11367,9,AR;AR50;AR75;ARM;AP;AP50;AP75;APL;ARL
4,RCV1 - Text Classification benchmarking,https://ai-strategies.org/ontology/ITO_14703,9,nDCG-at-5;nDCG-at-3;nDCG-at-1;Accuracy;P-at-1;...
...,...,...,...,...
2293,Tashkeela - Arabic Text Diacritization benchma...,https://ai-strategies.org/ontology/ITO_17254,1,Diacritic Error Rate
2294,CIFAR10-DVS - Event data classification benchm...,https://ai-strategies.org/ontology/ITO_17261,1,Accuracy
2295,Rotated MNIST - Rotated MNIST benchmarking,https://ai-strategies.org/ontology/ITO_17264,1,Test error
2296,StereoSet - Bias Detection benchmarking,https://ai-strategies.org/ontology/ITO_17269,1,ICAT Score


## Number of different metrics per benchmark (for those where any metrics are listed -- with topmost properties)


In [ ]:
result = graph.query("""
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix ito: <https://ai-strategies.org/ontology/>
SELECT  ?benchmark_label ?benchmark
       (COUNT(DISTINCT ?performance_measure) AS ?performance_measure_count)
       (GROUP_CONCAT(DISTINCT ?performance_measure_label;SEPARATOR=";") AS ?performance_measure_labels)
WHERE {
    {
        ?performance_measure rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    }
    UNION
    {
        ?performance_measure rdfs:subPropertyOf* ?performance_measure_topmost .
        ?performance_measure_topmost rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    }
    ?benchmark rdfs:subClassOf ito:Benchmarking ;
              rdfs:label ?benchmark_label ;
              rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure .
}
GROUP BY ?benchmark_label ?benchmark
ORDER BY DESC(?performance_measure_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number-of-different-metrics-per-benchmark-superproperties.csv', index=False)
df

,benchmark_label,benchmark,performance_measure_count,performance_measure_labels
0,COCO test-dev - Keypoint Detection benchmarking,https://ai-strategies.org/ontology/ITO_11334,10,Recall NEW;Precision;Average Path Length;Avera...
1,MSR-VTT - Video Retrieval benchmarking,https://ai-strategies.org/ontology/ITO_07273,10,R-at-K
2,"MIT-States, generalized split - Compositional ...",https://ai-strategies.org/ontology/ITO_07763,9,F1 score NEW;Area under curve (AUC) NEW;Accuracy
3,COCO test-challenge - Keypoint Detection bench...,https://ai-strategies.org/ontology/ITO_11367,9,Recall NEW;Precision;Average Path Length;Avera...
4,RCV1 - Text Classification benchmarking,https://ai-strategies.org/ontology/ITO_14703,9,Normalized Discounted Cumulative Gain;Accuracy...
...,...,...,...,...
2293,CollectMineralShards - Playing Starcraft II be...,https://ai-strategies.org/ontology/ITO_13661,1,Score
2294,ViZDoom Basic Scenario - Playing Game of Doom ...,https://ai-strategies.org/ontology/ITO_13663,1,Score
2295,Yahoo! Answers - Language Modelling benchmarking,https://ai-strategies.org/ontology/ITO_09832,1,Perplexity
2296,Yelp15 - Language Modelling benchmarking,https://ai-strategies.org/ontology/ITO_09834,1,Perplexity


## Total number of papers

In [ ]:
result = graph.query("""
SELECT (COUNT(?paper) as ?paper_count) WHERE {
    ?paper a edam:data_0971 .
}
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('total_number_of_papers.csv', index=False)
df

,paper_count
0,3883


## Number of papers that have at least one annotated metric

In [ ]:
result = graph.query("""
SELECT (COUNT(DISTINCT ?paper) as ?paper_count) WHERE {
    ?paper a edam:data_0971 . 
    ?benchmark_process_individual rdfs:seeAlso ?paper . 
    ?benchmark_process_individual ?performance_measure ?result .
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
}
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number-of-papers-that-have-at-least-one-annotated-metric.csv', index=False)
df

## Time span of publications covered by PWC

In [ ]:
result = graph.query("""
SELECT ?paper ?date WHERE {
    ?paper a edam:data_0971 .
    ?paper <http://purl.obolibrary.org/obo/date> ?date
}
ORDER BY ?date
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('time_span_of_publications_covered_by_PWC.csv', index=False)
df

,paper,date
0,https://ai-strategies.org/ontology/ITO_10000,2000-09-24
1,https://ai-strategies.org/ontology/ITO_16777,2003-07-18
2,https://ai-strategies.org/ontology/ITO_12116,2005-01-01
3,https://ai-strategies.org/ontology/ITO_15836,2006-05-12
4,https://ai-strategies.org/ontology/ITO_15832,2008-02-01
...,...,...
3866,https://ai-strategies.org/ontology/ITO_16523,2020-04-29
3867,https://ai-strategies.org/ontology/ITO_02413,2020-04-30
3868,https://ai-strategies.org/ontology/ITO_11960,2020-05-01
3869,https://ai-strategies.org/ontology/ITO_07230,2020-05-04


## Number of metrics aggregated by high-level process / task 

In [ ]:
result = graph.query("""
SELECT ?high_level_process_label ?high_level_process
       (COUNT(?subprocess_performance_measure) AS ?subprocess_performance_measure_count)
WHERE {
    ?high_level_process rdfs:subClassOf ito:Process ;
                        rdfs:label ?high_level_process_label . 

    ?subprocess rdfs:subClassOf* ?high_level_process .

    ?subprocess rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?subprocess_performance_measure .
    ?subprocess_performance_measure rdfs:subPropertyOf* ito:performance_measure .
}
GROUP BY ?high_level_process
ORDER BY DESC(?subprocess_performance_measure_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number_of_metrics_aggregated_by_high_level_process.csv', index=False)
df

,high_level_process_label,high_level_process,subprocess_performance_measure_count
0,Benchmarking,https://ai-strategies.org/ontology/Benchmarking,3665
1,Vision process,https://ai-strategies.org/ontology/ITO_00101,1948
2,Natural Language Processing,https://ai-strategies.org/ontology/ITO_00141,802
3,Fundamental AI process,https://ai-strategies.org/ontology/ITO_00115,352
4,Graph process,https://ai-strategies.org/ontology/ITO_00137,272
5,Biomedical process,https://ai-strategies.org/ontology/ITO_00126,221
6,Miscellaneous process,https://ai-strategies.org/ontology/ITO_00113,184
7,Segmentation process,https://ai-strategies.org/ontology/ITO_00478,174
8,Time Series process,https://ai-strategies.org/ontology/ITO_00131,126
9,Robotics process,https://ai-strategies.org/ontology/ITO_00600,90


## Number of benchmarks for each high-level process


In [ ]:
result = graph.query("""
SELECT ?high_level_process_label ?high_level_process
       (COUNT(?benchmark) AS ?benchmark_count)
WHERE {
    ?high_level_process rdfs:subClassOf ito:Process ;
                        rdfs:label ?high_level_process_label . 

    ?benchmark rdfs:subClassOf ito:Benchmarking . 
    ?benchmark rdfs:subClassOf* ?high_level_process .
}
GROUP BY ?high_level_process
ORDER BY DESC(?benchmark_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number_of_benchmarks_per_high_level_process.csv', index=False)
df

,high_level_process_label,high_level_process,benchmark_count
0,Benchmarking,https://ai-strategies.org/ontology/Benchmarking,2298
1,Vision process,https://ai-strategies.org/ontology/ITO_00101,1200
2,Natural Language Processing,https://ai-strategies.org/ontology/ITO_00141,491
3,Fundamental AI process,https://ai-strategies.org/ontology/ITO_00115,267
4,Graph process,https://ai-strategies.org/ontology/ITO_00137,161
5,Biomedical process,https://ai-strategies.org/ontology/ITO_00126,131
6,Segmentation process,https://ai-strategies.org/ontology/ITO_00478,111
7,Time Series process,https://ai-strategies.org/ontology/ITO_00131,91
8,Miscellaneous process,https://ai-strategies.org/ontology/ITO_00113,78
9,Playing Games,https://ai-strategies.org/ontology/ITO_00873,73


In [ ]:
! zip results.zip *.csv 

## Code for generating JSON for hierarchy visualization in d3js

In [ ]:
def class_to_dict(myclass):
    if myclass == ITO.Benchmarking:
        mydict = dict()
        mydict['name'] = "Benchmarking"
        mydict['size'] = 1
        return mydict
    mydict = dict()
    try:
        mydict['name'] = myclass.label[0]
    except IndexError:
        mydict['name'] = 'Class'
    children = list()
    for subclass in myclass.subclasses():
        children.append(class_to_dict(subclass))
    if len(children) > 0:
        mydict['children'] = children
    mydict['value'] = 1
    return mydict

In [ ]:
mydict = class_to_dict(ITO.Process)

In [ ]:
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(mydict, f, indent=2)

In [ ]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark) AS ?benchmark_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?performance_measure_subproperty rdfs:subPropertyOf* ?performance_measure .
    ?benchmark rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure_subproperty
}
GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('suproperty_statistics_per_benchmark.csv', index=False)
df

,performance_measure_label,performance_measure,benchmark_count
0,Accuracy,https://ai-strategies.org/ontology/ITO_01157,874
1,Precision,https://ai-strategies.org/ontology/ITO_04547,368
2,F1 score NEW,https://ai-strategies.org/ontology/ITO_00001,307
3,R-at-K,https://ai-strategies.org/ontology/ITO_00082,143
4,Area under curve (AUC) NEW,https://ai-strategies.org/ontology/ITO_00006,125
...,...,...,...
183,Average Treatment Effect Error,https://ai-strategies.org/ontology/ITO_15752,1
184,Mean target overlap ratio,https://ai-strategies.org/ontology/ITO_15853,1
185,Chamfer Distance,https://ai-strategies.org/ontology/ITO_15903,1
186,MACE,https://ai-strategies.org/ontology/ITO_15994,1
